# Devoir 2

Dans ce notebook, vous allez (1) vous familiariser d'avantage avec la méthode d'Euler pour résoudre numériquement des équations différentielles, et (2) utiliser cette méthode pour intégrer (résoudre) les équations du modèle FitzHugh-Nagumo pour déterminer le seuil de décharge du modèle.

###  1: La méthode d'Euler

Cette semaine, nous avons vu comment intégrer une équation différentielle avec la méthode d'Euler. De manière générale, un peut formuler une équation différentielel ordinaire de cette forme, où $x=x(t)$ est notre variable dynamique (dont nous voulons connaître l'évolution) :

$$ \frac{dx}{dt} = f(x,t)  $$

L'exemple utilisé dans le cours était assez simple, pour qu'on puisse comparer la solution analytique exacte au résultat de notre intégration numérique, nécessairement approximative. Avec $f(x,t) = -x/\tau$ (ici, $\tau$ est une constante de temps), et une valeur initiale $x(t=0)=x_0$, la solution de notre équation différentielle, c'est-à-dire l'évolution de notre système dynamique, est donnée par $x(t)=x_0 e^{-t/\tau}$. 

Cependant, la fonction $f(x,t)$, qui gouverne l'évolution de $x(t)$ peut être plus compliquée et par exemple dépendre explicitement du temps, par ex. quand une force externe agit sur le système. On va regarder un tel cas dans cette exercise.  

#### Exercise 1.1 

Intégrez numériquement l'équation différentielle 

$$ \frac{dx}{dt} = \cos(\omega t) $$

avec la condition initiale $x(t=0)=0$ et pour la fréquence d'oscillation $\omega = 2\pi$. Si vous connaissez la solution analytique, plottez-là ensemble avec votre solution numérique. Dans ce cas, la dérivée temporelle de $x(t)$ est donc donnée par la fonction $f(x,t)=\cos(\omega t) $. 

**Attention :** Veillez à vérifier que le pas de temps que vous utilisez est suffisamment petite pour que l'intégration numérique est correcte. 

**Rappel :** La valeur de $\pi$ est accessible dans la variable `np.pi`. 


In [ ]:
# Import all necessary modules
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# pi = np.pi

In [ ]:
# Choose appropriate discretization step for time
T = 3. # time until which the dynamics should be integrated
       # (not the time step!)
    
# (your code)

In [ ]:
# Define the function that returns the time derivative of x(t)

# (your code)

In [ ]:
# Determine iteratively the value of x for discrete times

# (your code)

In [ ]:
# Plot x(t) against t
# (If you know the theoretical solution, plot that one, too)

#### Exercise 1.2

Répétez l'intégration pour deux autres valeurs de $\omega$ et plottez les différentes solutions ensemble.

In [ ]:
# Repeat for lower oscillation freqency
omega1 = np.pi

# (your code)

In [ ]:
# Repeat for increased oscillation freqency
omega2 = 4*np.pi

# (your code)

#### Exercise 1.3

Intégrez numériquement l'équation différentielle suivante :

$$ \frac{dx}{dt} = - x + a \cos(\omega t) $$

avec la condition initiale $x(t=0)=10$ et avec les paramètres  $\omega = 2\pi$ et $a = 1$.

In [ ]:
# Define the time step etc.

# (your code)

# Define the function that returns the derivative

# (your code)

# Solve iteratively for x(x)

# (your code)


In [ ]:
# Plot the numerically deterimined solution together with the solution
# in the undamped case (Exercise 1.1)

# (your code)

### 2: Le modèle de FitzHugh-Nagumo

Dans le TD de cette semaine, nous avons étudié le modèle FitzHugh-Nagumo, qui décrit de manière simplifiée la décharge neuronale. Le modèle est spécifié par les équations 


$$ \frac{dV}{dt} = V - V^3/3 - W + I ,$$

$$ \frac{dW}{dt} = 0.08 (V+ 0.7 - 0.8 W), $$


Nous avons constaté qu'en fonction du courant $I$ injecté, le neurone décharge (génére des pics de potentiel $V$) ou reste silencieux (les variables ne varient pas dans le temps, après une rélaxation intitiale). 

#### Exercise 2.1

Déterminez, à deux décimales près, la valeur seuil de $I$ au-dessus de laquelle le neurone décharge de manière répétée. Pour éviter des PA singulier uniquement dus aux conditions initiales, choisissez des valeurs initiales $V(t=0)=-1$, $W(t=0)=0$.

**Astuce :** Vous pouvez réutilisez le code du TD pour l'implémentation du modèle. 

In [ ]:
# Time discretization
dt = 0.01
T = 200.0
t = np.arange(0,T,dt)
nt = len(t)

# Definition of derivative fonctions that describe 
# the dynamics of the model variables
def dVdt(v,w,I):
    return v-(v**3)/3-w+I

def dWdt(v,w):
    return 0.08*(v+0.7-0.8*w)

# (your code)

#### Exercise 2.2

Pour une valeur de $I$ au-dessus du seuil, c'est-à-dire quand le neurone décharge, plottez le potentiel de membrane $V$ contre la valeur de la variable lente $W$. Cela est appelé un "phase plot", est permet de caractériser comment les deux variables co-évoluent. 

In [ ]:
# (your code)